# A Very Brief Intoduction to Using STORM2

## Connecting to STORM2

```bash
$ ssh <user>@storm2-fe.lancs.ac.uk
```

## Copying files to STORM2 using rsync

In the following replace \<local file\> with the name of the file you want to copy, \<user\> with your storm2 user name, and \<remote directory\> with the name of the directory you wish to copy it to on storm2. 

```bash
$ rsync -pavx --progress <local file> <user>@storm2-fe.lancs.ac.uk:<remote file>
```

### Useful resources

[More information on how to use rsync](https://www.digitalocean.com/community/tutorials/how-to-use-rsync-to-sync-local-and-remote-directories)

## Copying files to and from STORM2 using scp

### Copying a file to STORM2

```bash
$ scp <local file> <user>@storm2-fe.lancs.ac.uk:<remote file>
```

### Copying a file from STORM2


```bash
$ scp <user>@storm2-fe.lancs.ac.uk:<remote file> <local file> 
```

## Installing, configuring and using pyenv

### Installing pyenv

```shell
$ git clone https://github.com/pyenv/pyenv.git ./.pyenv
$ echo -e "export PATH=\"\$HOME/.pyenv/bin:\$PATH\"\neval \"\$(pyenv init --path)\"\n" > start-pyenv
$ chmod +x start-pyenv
```

### Starting pyenv

```shell
$ cd ~
$ source ./start-pyenv
```

### Listing available python systems

```bash
$ pyenv install --list
```

### Installing a python system 

The following example installs CPython version 3.9.16

```bash
$ pyenv install 3.9.16
```

### Listing installed python systems

```bash
$ pyenv versions
```

### Changing the selected python system

In the following example, the working python system is set to CPython version 3.9.16

```bash
$ pyenv global 3.9.1
```

### Using the currently selected python system

```bash
$ python 
```

## Virtual python environments

The following assumes you have installed and started pyenv

### Creating a new environment

```bash
$ mkdir project-1
$ cd project-1
$ python -m venv env
```

### Starting an existing environment

```bash
$ cd project-1
$ source ./env/bin/activate
```

### Stop a running environment

```bash
$ deactivate
```

### Removing an environment

```bash
$ rm -rf project-1
```

### Useful resources

[An introduction to python virtual environments](https://python.land/virtual-environments/virtualenv)

# Using slurm

### Prerequisites

1. Start pyenv
2. If you are using a virtual python environment, activate it 

### Example 1 - Simple python script

Create the following two files in your home directory on storm2

#### sleep.py

```python
import sys
import time
import datetime

print(sys.version)
t = int(sys.argv[1])
print("the time is now ",datetime.datetime.now())
print("zzzzzz")
time.sleep(t)
print("hello - I slept for ",t," seconds")
print("the time is now ",datetime.datetime.now())
```

#### sleep.sh

```shell
#!/bin/bash

#SBATCH -J sleep1-job
#SBATCH -c 1
#SBATCH -o sleep-1-job.out
#SBATCH --mail-user=dan.grose@lancaster.ac.uk

source ~/start-pyenv

srun python sleep.py 120
```

#### Submit the job

```bash
$ sbatch sleep.sh
```

#### Monitoring the job

```bash
$ squeue
```

#### Examining results sent to stdout

```bash
$ head sleep-1-job.out
```

#### Adding more processors

To make more processors avaiable for your job increase the value specified in the line

```shell
#SBATCH -c 1
```

in the sleep.sh script.

### Example 2 - Simple R script

Create the following two files in your home directory on storm2

#### sleep.R

```R
args <- commandArgs(trailingOnly = TRUE)
t <- as.double(args)
cat("the time is now ",as.character(Sys.time()),'\n')
cat("zzzzzz",'\n')
Sys.sleep(t)
cat("hello - I slept for ",t," seconds",'\n')
cat("the time is now ",as.character(Sys.time()),'\n')
```

#### sleep.sh

```shell
#!/bin/bash

#SBATCH -J sleep1-job
#SBATCH -c 1
#SBATCH -o sleep-1-job.out
#SBATCH --mail-user=dan.grose@lancaster.ac.uk

srun Rscript sleep.R 120
```

#### Submit the job

```bash
$ sbatch sleep.sh
```

#### Monitoring the job

```bash
$ squeue
```

#### Examining results sent to stdout

```bash
$ head sleep-1-job.out
```

#### Adding more processors

To make more processors avaiable for your job increase the value specified in the line

```shell
#SBATCH -c 1
```

in the sleep.sh script.

### Example 3 - Simple python job array 

```shell
#!/bin/bash

#SBATCH -J sleep-array-job
#SBATCH --mail-user=dan.grose@lancaster.ac.uk
#SBATCH --mail-type=ALL
#SBATCH --array=10-120:10
#SBATCH -c 1
#SBATCH -o sleep-1-job_%A_%a.out

source ~/start-pyenv
echo ARRAY ID IS : $SLURM_ARRAY_TASK_ID
srun python sleep.py $SLURM_ARRAY_TASK_ID
```

# Job Management

## squeue

You can tailor the information that **squeue** provides.

#### Example 1
Get all of the default information plus the number of cpus that are being used by a particular job.

```bash
$ squeue  -o "%.18i %.9P %.20j %.8u %.2t %.10M %.6D %R %C"
```

Notice how this was done by specifying a **formatting string** in conjunction with the **-o** option. Also notice that the size of the fields for each output can be set using a **.n** modifier.

For more information on the **-o** option and **formatting string** see  the [slurm reference card](https://slurm.schedmd.com/pdfs/summary.pdf).

## sinfo

**sinfo** can be used to get information about the nodes. There are lots of options which (like with squeue) can be set using the **-o** option and a **formatting string**. 

#### Example 1

Get information about used and available cores on each node.

```bash
$ sinfo -o "%n %C"
```

The output shows 


**A** (allocated) the number of cores on the node that have been allocated to jobs (i.e. are being used) \
**I** (idle) the number of cores on the node not being used i.e the number available for use \
**O** (other) number of cores on the node that are offline or in some other unusual state - should be 0 most of the time \
**T** (total) the total number of cores on the node

## htop

Once you have information regarding which nodes your jobs are running on, you can have a look at how much memory and cpu percentage it is using with the **htop** command. However, **htop** only gives information regarding the node on which the command was submitted. So, for it to be useful, you fist have to login into the node where your job is running using **ssh**. Note that you should do this from your **storm2-fe** shell prompt. Also note, that becuase you are using **ssh** on **storm2** then you do not need to re-enter your password.

#### Example 1

Imagine you have used **squeue** and determined that your job is running on **dantzig**. To get detailed information regarding cpu and memory use for your jobs on **dantzig** use

```bash
$ ssh dantzig
$ htop -u <username>
```

where **<username>** is your **storm2** user name.

### Useful resources

[slurm reference card](https://slurm.schedmd.com/pdfs/summary.pdf)